In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [3]:
cmm_data = pd.read_csv('/home/ruggiec/Downloads/pharmacy_tx.csv')

#cmm_data_no_dups = cmm_data.drop_duplicates()

cmm_train, cmm_test = train_test_split(cmm_data, shuffle=True, 
                             random_state= 614, test_size=0.2)

In [4]:
def is_it_generic(text):
    if text.split()[0] == 'generic':
        return 1
    else:
        return 0
    
def drug_name(text):
    return text.split()[1]

def pharm_split(text):
    return int(text.split('#')[1])

In [5]:
cmm_train['tx_date'] = pd.to_datetime(cmm_train.tx_date)
cmm_train['month'] = cmm_train.tx_date.dt.month
cmm_train['day_of_year'] = cmm_train.tx_date.dt.dayofyear

In [6]:
cmm_train['generic'] = cmm_train.drug.apply(is_it_generic)
cmm_train['name'] = cmm_train['drug'].apply(drug_name)
cmm_train['private'] = cmm_train.group.isna().astype(int)
cmm_train['popularity'] = cmm_train.groupby('name')[['tx_date']].transform('count')
cmm_train['rejected'] = cmm_train['rejected'].astype(int)
cmm_train['rejected_count'] = cmm_train.groupby('name')['rejected'].transform('sum')
cmm_train['pharm_num'] = cmm_train.pharmacy.apply(pharm_split)

In [ ]:
cmm_train['binpcn'] = cmm_train.bin.astype(str).fillna('') + cmm_train.pcn.fillna('')
cmm_train['bingroup'] = cmm_train.bin.astype(str).fillna('') + cmm_train.group.fillna('')
cmm_train['binpcngroup'] = cmm_train.bin.astype(str).fillna('') + cmm_train.pcn.fillna('') + cmm_train.group.fillna('')

In [ ]:
print('There are', cmm_train.binpcngroup.nunique(), 
      'separate insurance plans in the data set.')
print('There are', cmm_train.drug.nunique(), 
      'drugs in the data set.')
print('Leading to', cmm_train.binpcngroup.nunique()*cmm_train.drug.nunique(), 
      'combinations of drug and insurance plan.')

In [ ]:
cmm_train.binpcngroup.sample(1).values[0]

In [ ]:
cmm_train[cmm_train.binpcngroup == 
          cmm_train.binpcngroup.sample(1).values[0]].groupby('drug')['patient_pay'].unique()

In [ ]:
cmm_train[cmm_train.binpcngroup == 
          cmm_train.binpcngroup.sample(1).values[0]].groupby('drug')['patient_pay'].std()